In [2]:
import pandas as pd
import numpy as np

In [37]:
#读取数据，第一行作为columns，跳过第二行的中文说明字段
original_data=pd.read_csv("ecommerce_user_behavior_dataset.csv",header=0,skiprows=[1],encoding="gbk")
original_data.head()

0       38037
1      103986
2      101942
3       71612
4       49725
        ...  
995    104162
996     99003
997     72395
998     59758
999     74312
Name: Income, Length: 1000, dtype: int64


构建RFM模型，需要界定R(Recency),F(Frequency),M(Monetary)，分别衡量最近一次消费是在几天前？消费的频率如何？消费金额如何？其与客户价值的关系分别为，R为负相关，F为正相关，M为正相关。为便于处理，我们新定义一个r来替代R，满足r=-R，这样RFM三个因子都为正相关。

In [ ]:
#实现自定义对多个列指标进行加权以构成核心R,F,M因子
#考虑到不同的指标间的数据量级差别较大，这里对namelist中的每个值都进行对数化处理
def composit_means(namelist,weight_list,group):
    """
    namelist是需要进行加权的列名列表
    weight_list是各个列名中的值对应的权重(索引一一对应,即namelist[a]的元素对应的权重为weight_list)
    group是你需要进行自定义加权处理的数据集，需要在分组后进行使用
    """
    if len(namelist)!=len(weight_list):
        Error1="Error,check the whether namelist and weight list has the same dimensions"
        return Error1
    if sum(weight_list)!=1:
        Error2="Error, check whether your weight_list has a sum of 1"
        return Error2
    weightedsum=0
    for name, weight in zip(namelist,weight_list):
        weightedsum+=np.log(group[name].mean())*weight
    return weightedsum

In [ ]:
#进行RFM计算（这里注意下agg语法的书写要求）
#agg的语法为，输入一个字典，键为列名，值为计算方法，常见的有"count" "sum" "mean"······
RFM=original_data.groupby("User_ID").agg({"Last_Login_Days_Ago": "mean","Purchase_Frequency":"sum","Average_Order_Value":"sum"
                                          }).rename(columns={"Last_Login_Days_Ago":"Recency","Purchase_Frequency":"Frequency"
                                                            ,"Average_Order_Value":"Monetary"})
RFM["Recency"]=-RFM["Recency"]
print(RFM)

         Recency  Frequency  Monetary
User_ID                              
#1          -5.0          7        18
#10        -13.0          8        61
#100        -9.0          7        92
#1000      -12.0          7       175
#101       -29.0          5        51
...          ...        ...       ...
#995       -10.0          1        60
#996       -18.0          7        83
#997       -16.0          3       180
#998        -8.0          8       130
#999        -1.0          3       152

[1000 rows x 3 columns]


In [ ]:
#下面是对自定义加权的测试
Mlist=["Income","Total_Spending","Average_Order_Value"]
weight=[0.2,0.3,0.5]
#计算RFM,这里用apply是因为agg返回的是series，无法支持我们进行自定义计算。
RFM_test = original_data.groupby("User_ID").apply(
    lambda group: pd.Series({
        "Recency": group["Last_Login_Days_Ago"].mean(),
        "Frequency": group["Purchase_Frequency"].sum(),
        "Monetary": composit_means(Mlist, weight, group)
    })
)

RFM_test["Recency"]=-RFM_test["Recency"]
print(RFM_test)

         Recency  Frequency  Monetary
User_ID                              
#1          -5.0        7.0  5.907132
#10        -13.0        8.0  6.952240
#100        -9.0        7.0  6.878207
#1000      -12.0        7.0  6.840940
#101       -29.0        5.0  6.087967
...          ...        ...       ...
#995       -10.0        1.0  5.877430
#996       -18.0        7.0  6.442719
#997       -16.0        3.0  6.716892
#998        -8.0        8.0  6.614837
#999        -1.0        3.0  6.795933

[1000 rows x 3 columns]


/tmp/ipykernel_17378/1526713920.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  RFM_test = original_data.groupby("User_ID").apply(


实际上，这里由于User ID和对应的变量间形成了单射，所以RFM这个dataframe完全可以就直接切片，采用agg的形式是为了便于以后优化（如合并多个因子后进行归类。）

In [ ]:
#进行简单归类
#评分
RFM["R_score"]=pd.qcut(RFM["Recency"],q=5,labels=[1,2,3,4,5])
RFM["F_score"]=pd.qcut(RFM["Frequency"],q=5,labels=[1,2,3,4,5])
RFM["M_score"]=pd.qcut(RFM["Monetary"],q=5,labels=[1,2,3,4,5])
#